In [7]:
import findspark
findspark.init()

from pyspark import SparkContext

from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.functions import*
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType, DoubleType

import pandas as pd
import json
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

# For “Credit Card System,” create a Python and PySpark SQL program to read/extract the following JSON files according to the specifications found in the mapping document.


In [80]:
def read_data():
    file=input("Which file would you like to extract? Note: Please include file extension")
    while ".json" not in file:
        print("Please make sure this is a json file and include the file extension. Enter again: ")
        file=input("Which file would you like to extract? Note: Please include file extension")
    spark = SparkSession.builder.master("local[*]").appName(file).getOrCreate()
    df = spark.read.json(file)
    print("Extraction Completed")
    if "branch" in file:
        df = df.na.fill(value=99999, subset=["BRANCH_ZIP"])
        df = df.withColumn("PARTIAL_PHONE", f.concat(f.lit("("), f.col("BRANCH_PHONE")[1:3], f.lit(")")))
        df = df.withColumn("BRANCH_PHONE", f.concat(f.col("PARTIAL_PHONE"), f.col("BRANCH_PHONE")[4:3], f.lit("-"), f.col("BRANCH_PHONE")[7:9]))
        df = df.drop("PARTIAL_PHONE")
        global branch_df
        branch_df =df
    elif "custmer" in file:
        df = df.withColumn("FIRST_NAME", f.initcap(df.FIRST_NAME))
        df = df.withColumn("MIDDLE_NAME", f.lower(df.MIDDLE_NAME))
        df = df.withColumn("LAST_NAME", f.initcap(df.LAST_NAME))
        # Since we don't ave exact area codes, we are assuming an area code for now. We may change this in the future if we can have a dictionary mapping 
        # an area code kist to city or state list, and by checking CUST_STATE or CUST_CITY we can better guess who may have what area code (if they have mostly lived there)
        df = df.withColumn("CUST_PHONE", f.concat((f.lit("(917)")), f.col("CUST_PHONE")))
        df = df.withColumn("CUST_PHONE", f.concat(f.col("CUST_PHONE")[0:8], f.lit("-"), f.col("CUST_PHONE")[9:12]))
        df = df.withColumn("STREET_NAME,APT_NO", f.concat(f.col("STREET_NAME"), f.lit(","), f.col("APT_NO")))
        df = df.drop("APT_NO", "STREET_NAME")
        global customer_df
        customer_df = df
    elif "credit" in file:
        df = df.withColumn("DAY,MONTH,YEAR", f.concat(f.col("YEAR"), f.col("MONTH"), f.col("DAY")))
        df = df.drop("DAY", "MONTH", "YEAR")
        global credit_df
        credit_df = df
    print("Transformation Complete")

read_data()


23/03/02 08:59:14 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
Extraction Completed
Transformation Complete


In [82]:
def write_data():
    start_write=input("Would you like to load this data into MySQL DB? (y/n)")
    if start_write in ["yes", "y"]:
        print("Loading...")
        branch_df.write.format("jdbc").mode("append").option("url", "jdbc:mysql://localhost:3306/creditcard_capstone").option("dbtable", "creditcard_capstone.CDW_SAPP_BRANCH").option("truncate","true").option("user", "root").option("characterEncoding","UTF-8").option("useUnicode", "true").option("password", "password").save()
        customer_df.write.format("jdbc").mode("append").option("url", "jdbc:mysql://localhost:3306/creditcard_capstone").option("dbtable", "creditcard_capstone.CDW_SAPP_CUSTOMER").option("truncate","true").option("user", "root").option("characterEncoding","UTF-8").option("useUnicode", "true").option("password", "password").save()
        credit_df.write.format("jdbc").mode("append").option("url", "jdbc:mysql://localhost:3306/creditcard_capstone").option("dbtable", "creditcard_capstone.CDW_SAPP_CREDIT_CARD").option("truncate","true").option("user", "root").option("characterEncoding","UTF-8").option("useUnicode", "true").option("password", "password").save()
    print("Finished Writing And Loading Data to Database")

write_data()


Loading...


Finished Writing And Loading Data to Database
